In [8]:
import gzip # pour décompresser les données
import pickle 
import matplotlib.pyplot as plt # pour l'affichage
import torch,torch.utils.data
import numpy as np
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [9]:
# Recuperation des données

# nombre d'image lues à chaque fois dans la base d'apprentissage (laisser à 1 sauf pour la question optionnelle sur les minibatchs)
TRAIN_BATCH_SIZE = 1
# on charge les données de la base MNIST
data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='latin1')
# images de la base d'apprentissage
train_data = torch.Tensor(data[0][0])
# labels de la base d'apprentissage
train_data_label = torch.Tensor(data[0][1])
# images de la base de test
test_data = torch.Tensor(data[1][0])
# labels de la base de test
test_data_label = torch.Tensor(data[1][1])
# on crée la base de données d'apprentissage (pour torch)
train_dataset = torch.utils.data.TensorDataset(train_data,train_data_label)
# on crée la base de données de test (pour torch)
test_dataset = torch.utils.data.TensorDataset(test_data,test_data_label)
# on crée le lecteur de la base de données d'apprentissage (pour torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
# on crée le lecteur de la base de données de test (pour torch)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


In [6]:
# Partie 1 : Perceptron

weight = 0.5
nb_step = 0.001
w = np.full((785, 10), weight)

for d in range(len(train_data)):
    x = train_data[d].numpy()
    x = np.append(x, 1)
    y = np.dot(x, w)
    dl = train_data_label[d].numpy()
    a = nb_step * (dl - y)
    b = np.outer(a, x)
    w += np.transpose(b)

nbCorrect = 0
for d in range(len(test_data)):
    x = test_data[d].numpy()
    x = np.append(x, 1)
    y = np.dot(x, w)
    index = np.argmax(test_data_label[d])
    index2 = np.argmax(y)
    if index == index2:
        nbCorrect += 1
print(nbCorrect/len(test_data) * 100,"%")

83.67142857142858 %


In [5]:
# Partie 2 : Shallow network

weight = 0.5
nb_step = 0.001
w_h = np.full((785, 50), weight)
w_o = np.full((50, 10), weight)

for d in range(len(train_dataset)):

    # Choisir entrée
    x = train_data[d].numpy()
    x = np.append(x, 1)

    # Propagation yi(1)
    s_h = np.dot(x, w_h)
    e_h = np.exp(-s_h)
    y_h = 1 / (1 + e_h)

    # Propagation yi(2)
    y_o = np.dot(y_h, w_o)

    # Rétro Propagation 

    # Calcul de l'erreur
    dl = train_data_label[d].numpy()
    m_o = dl - y_o

    # Rétro Propa de l'erreur
    a1 = y_h * (1 - y_h)
    a2 = np.dot(m_o, np.transpose(w_o))
    m_h = a1 * a2

    # Modification poids
    dw_o = nb_step * np.outer(m_o, y_h)
    w_o += np.transpose(dw_o)
    dw_h = nb_step * np.outer(m_h, x)
    w_h += np.transpose(dw_h)

In [4]:
nbCorrect = 0
for d in range(len(test_data)):
    x = train_data[d].numpy()
    x = np.append(x, 1)

    s_h = np.dot(x, w_h)
    e_h = np.exp(-s_h)
    y_h = 1 / (1 + e_h)

    y_o = np.dot(y_h, w_o)
    index = np.argmax(test_data_label[d])
    index2 = np.argmax(y_o)
    if index == index2:
        nbCorrect += 1
print(nbCorrect/len(test_data) * 100,"%")

9.785714285714285 %


In [ ]:
#  Partie 3 : Deep network

In [ ]:
# Partie 4 : Pour aller plus loin

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx and log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Training settings

batch_size = 64
test_batch_size = 1000
epochs = 14
lr = 1.0
gamma = 0.7
no_cuda = False
seed = 1
log = False

use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()
